In [69]:
import json
import csv
import tweepy
import re
import nltk
from nltk.stem import WordNetLemmatizer
import pandas as pd
from nltk.corpus import stopwords
stopwords = set(stopwords.words('english'))
lemmatizer = WordNetLemmatizer()
import string
from nltk import word_tokenize

In [127]:
df = pd.read_csv('coronatweets.csv')
wn = nltk.WordNetLemmatizer()

In [128]:
df=df.drop(columns="Unnamed: 0")
df

,Tweet_text
0,Georgia may need to dip deep in its reserve fu...
1,Vatican: Migrants Are the ‘True Victims’ of th...
2,"Coronavirus outbreak: Trudeau praises ""positiv..."
3,NOOOOOOO! No corona virus vaccine has EVER bee...
4,#HESA is meeting via videoconference today at ...
...,...
24995,#Covid19 Ramaphosa says we must find those who...
24996,#Breastfeeding and #COVID19 guidelines algorit...
24997,This is the article you should read today. #CO...
24998,I thank @GovWike for extending the lockdown da...


In [129]:
df=df.drop_duplicates()
df=df.reset_index()
df=df.drop(columns='index')

In [130]:
df['Tweet_text'] = df['Tweet_text'].astype(str)
df

,Tweet_text
0,Georgia may need to dip deep in its reserve fu...
1,Vatican: Migrants Are the ‘True Victims’ of th...
2,"Coronavirus outbreak: Trudeau praises ""positiv..."
3,NOOOOOOO! No corona virus vaccine has EVER bee...
4,#HESA is meeting via videoconference today at ...
...,...
22526,My 1 year old is obsessed with @TheRealFunyuns...
22527,#Covid19 Ramaphosa says we must find those who...
22528,This is the article you should read today. #CO...
22529,I thank @GovWike for extending the lockdown da...


In [131]:
def clean_text(text):
    text_lc = "".join([word.lower() for word in text if word not in string.punctuation]) # remove puntuation
    text_rc = re.sub('[0-9]+', '', text_lc)
    nopunc = re.sub('((www\.[^\s]+)|(https?://[^\s]+)|(http?://[^\s]+))', '', text_rc) #remove urls
    nopunc = re.sub(r'http\S+', '', nopunc) #remove usernames
    nopunc = re.sub(r'#([^\s]+)', r'\1', nopunc) #remove hashtags
    nopunc= re.sub(r'(.)\1+', r'\1\1', nopunc)  #remove repeating characters 
    tokens = re.split('\W+', nopunc)    # tokenization
    text = [word for word in tokens if word not in stopwords]  # remove stopwords and stemming
    return text
df['Text_cleaned'] = df['Tweet_text'].apply(lambda x: clean_text(x))


In [132]:
def lemmatizer(text):
    text = [wn.lemmatize(word) for word in text]
    return text
df['Text_lemmatized'] = df['Text_cleaned'].apply(lambda x: lemmatizer(x))

def listtostr(s):
    str1=" "
    return (str1.join(s))
df['Text_lemmatized_str'] = df['Text_lemmatized'].apply(lambda x: listtostr(x))
df.head()

,Tweet_text,Text_cleaned,Text_lemmatized,Text_lemmatized_str
0,Georgia may need to dip deep in its reserve fu...,"[georgia, may, need, dip, deep, reserve, fund,...","[georgia, may, need, dip, deep, reserve, fund,...",georgia may need dip deep reserve fund balance...
1,Vatican: Migrants Are the ‘True Victims’ of th...,"[vatican, migrants, true, victims, coronavirus...","[vatican, migrant, true, victim, coronavirus, ...",vatican migrant true victim coronavirus pandemic
2,"Coronavirus outbreak: Trudeau praises ""positiv...","[coronavirus, outbreak, trudeau, praises, posi...","[coronavirus, outbreak, trudeau, praise, posit...",coronavirus outbreak trudeau praise positive c...
3,NOOOOOOO! No corona virus vaccine has EVER bee...,"[noo, corona, virus, vaccine, ever, safe, kill...","[noo, corona, virus, vaccine, ever, safe, kill...",noo corona virus vaccine ever safe kill need v...
4,#HESA is meeting via videoconference today at ...,"[hesa, meeting, via, videoconference, today, p...","[hesa, meeting, via, videoconference, today, p...",hesa meeting via videoconference today pm et w...


In [133]:
wordfreq = {}
for sentence in df['Text_lemmatized_str']:
    tokens = nltk.word_tokenize(sentence)
    for token in tokens:
        if token not in wordfreq.keys():
            wordfreq[token] = 1
        else:
            wordfreq[token] += 1

In [138]:
import operator
sorted_wordfreq=sorted(wordfreq.items(),key=operator.itemgetter(1),reverse=True)
sorted_wordfreq

[('coronavirus', 13231),
 ('covid', 11496),
 ('pandemic', 1924),
 ('people', 1910),
 ('u', 1781),
 ('new', 1758),
 ('death', 1644),
 ('virus', 1589),
 ('case', 1538),
 ('amp', 1463),
 ('via', 1308),
 ('trump', 1189),
 ('may', 1154),
 ('time', 1096),
 ('get', 1084),
 ('home', 1053),
 ('say', 1041),
 ('health', 1018),
 ('corona', 1015),
 ('state', 985),
 ('like', 976),
 ('one', 948),
 ('news', 935),
 ('lockdown', 928),
 ('need', 917),
 ('help', 904),
 ('day', 852),
 ('response', 823),
 ('today', 716),
 ('work', 701),
 ('know', 699),
 ('would', 681),
 ('go', 676),
 ('world', 670),
 ('test', 664),
 ('due', 662),
 ('government', 660),
 ('see', 656),
 ('update', 631),
 ('make', 626),
 ('country', 625),
 ('could', 625),
 ('crisis', 622),
 ('business', 614),
 ('care', 612),
 ('uk', 608),
 ('many', 608),
 ('back', 591),
 ('life', 583),
 ('american', 579),
 ('year', 562),
 ('mask', 557),
 ('take', 549),
 ('want', 544),
 ('vaccine', 533),
 ('going', 531),
 ('even', 529),
 ('way', 524),
 ('public'

In [139]:
len(sorted_wordfreq)

37448

In [140]:
import heapq
most_freq = heapq.nlargest(25000, wordfreq, key=wordfreq.get)

In [141]:
sentence_vectors = []
for sentence in df['Text_lemmatized_str']:
    sentence_tokens = nltk.word_tokenize(sentence)
    sent_vec = []
    for token in most_freq:
        if token in sentence_tokens:
            sent_vec.append(1)
        else:
            sent_vec.append(0)
    sentence_vectors.append(sent_vec)

In [143]:
file1 = open('positive_words.txt','r')
file2 = open('negative_words.txt','r')
positive_words = file1.read().split('\n')
negative_words = file2.read().split('\n')

In [144]:
positive_words_from_dataset = []
negative_words_from_dataset = []
neutral_words_from_dataset = []

for word in wordfreq.keys():
    if word in positive_words:
        positive_words_from_dataset.append(word)
    elif word in negative_words:
        negative_words_from_dataset.append(word)
    else:
        neutral_words_from_dataset.append(word)

In [145]:
len(positive_words_from_dataset) #positive BOW

835

In [146]:
len(negative_words_from_dataset) #negative BOW

1711

In [147]:
len(neutral_words_from_dataset) #Neutral BOW

34902

In [149]:
positive_words_from_dataset

['praise',
 'positive',
 'safe',
 'survivor',
 'wise',
 'important',
 'enough',
 'progress',
 'top',
 'amazing',
 'best',
 'glad',
 'like',
 'smarter',
 'hot',
 'work',
 'benefit',
 'freed',
 'patient',
 'recover',
 'bright',
 'significant',
 'win',
 'trump',
 'protection',
 'wonder',
 'promise',
 'loved',
 'right',
 'modern',
 'great',
 'helping',
 'cute',
 'support',
 'clearly',
 'powerful',
 'rapture',
 'revelation',
 'wow',
 'relief',
 'creative',
 'protect',
 'advantage',
 'panoramic',
 'love',
 'desirable',
 'regard',
 'recovery',
 'lead',
 'enjoy',
 'smooth',
 'proper',
 'better',
 'well',
 'handsome',
 'interesting',
 'balanced',
 'worked',
 'worthy',
 'pride',
 'accurate',
 'tremendously',
 'stable',
 'good',
 'luxury',
 'available',
 'freedom',
 'strong',
 'tough',
 'appreciate',
 'supporting',
 'exceed',
 'affordable',
 'cure',
 'breakthrough',
 'genius',
 'faith',
 'thank',
 'clear',
 'fabulous',
 'free',
 'beloved',
 'extraordinary',
 'fair',
 'healthy',
 'appeal',
 'fines

In [152]:
def sentiment_score(text):
    sent_score=0
    for word in text:
        if word in positive_words_from_dataset:
            sent_score+=1
        elif word in negative_words_from_dataset:
            sent_score-=1
    return sent_score

df['sentiment_score'] = df['Text_lemmatized'].apply(lambda x: sentiment_score(x))
df.head()           

,Tweet_text,Text_cleaned,Text_lemmatized,Text_lemmatized_str,sentiment_score
0,Georgia may need to dip deep in its reserve fu...,"[georgia, may, need, dip, deep, reserve, fund,...","[georgia, may, need, dip, deep, reserve, fund,...",georgia may need dip deep reserve fund balance...,0
1,Vatican: Migrants Are the ‘True Victims’ of th...,"[vatican, migrants, true, victims, coronavirus...","[vatican, migrant, true, victim, coronavirus, ...",vatican migrant true victim coronavirus pandemic,0
2,"Coronavirus outbreak: Trudeau praises ""positiv...","[coronavirus, outbreak, trudeau, praises, posi...","[coronavirus, outbreak, trudeau, praise, posit...",coronavirus outbreak trudeau praise positive c...,1
3,NOOOOOOO! No corona virus vaccine has EVER bee...,"[noo, corona, virus, vaccine, ever, safe, kill...","[noo, corona, virus, vaccine, ever, safe, kill...",noo corona virus vaccine ever safe kill need v...,-3
4,#HESA is meeting via videoconference today at ...,"[hesa, meeting, via, videoconference, today, p...","[hesa, meeting, via, videoconference, today, p...",hesa meeting via videoconference today pm et w...,-1


In [154]:
df.loc[df['sentiment_score'] > 0 ,'Label'] = 'Positive'
df.loc[df['sentiment_score'] < 0 ,'Label'] = 'Negative'
df.loc[df['sentiment_score'] == 0 ,'Label'] = 'Neutral'


In [155]:
df

,Tweet_text,Text_cleaned,Text_lemmatized,Text_lemmatized_str,sentiment_score,Label
0,Georgia may need to dip deep in its reserve fu...,"[georgia, may, need, dip, deep, reserve, fund,...","[georgia, may, need, dip, deep, reserve, fund,...",georgia may need dip deep reserve fund balance...,0,Neutral
1,Vatican: Migrants Are the ‘True Victims’ of th...,"[vatican, migrants, true, victims, coronavirus...","[vatican, migrant, true, victim, coronavirus, ...",vatican migrant true victim coronavirus pandemic,0,Neutral
2,"Coronavirus outbreak: Trudeau praises ""positiv...","[coronavirus, outbreak, trudeau, praises, posi...","[coronavirus, outbreak, trudeau, praise, posit...",coronavirus outbreak trudeau praise positive c...,1,Positive
3,NOOOOOOO! No corona virus vaccine has EVER bee...,"[noo, corona, virus, vaccine, ever, safe, kill...","[noo, corona, virus, vaccine, ever, safe, kill...",noo corona virus vaccine ever safe kill need v...,-3,Negative
4,#HESA is meeting via videoconference today at ...,"[hesa, meeting, via, videoconference, today, p...","[hesa, meeting, via, videoconference, today, p...",hesa meeting via videoconference today pm et w...,-1,Negative
...,...,...,...,...,...,...
22526,My 1 year old is obsessed with @TheRealFunyuns...,"[year, old, obsessed, therealfunyuns, fritolay...","[year, old, obsessed, therealfunyuns, fritolay...",year old obsessed therealfunyuns fritolay get ...,2,Positive
22527,#Covid19 Ramaphosa says we must find those who...,"[covid, ramaphosa, says, must, find, infected,...","[covid, ramaphosa, say, must, find, infected, ...",covid ramaphosa say must find infected early p...,-2,Negative
22528,This is the article you should read today. #CO...,"[article, read, today, covid, homelessness, nyc]","[article, read, today, covid, homelessness, nyc]",article read today covid homelessness nyc,0,Neutral
22529,I thank @GovWike for extending the lockdown da...,"[thank, govwike, extending, lockdown, date, ob...","[thank, govwike, extending, lockdown, date, ob...",thank govwike extending lockdown date obioakpo...,1,Positive


In [156]:
df.loc[df['Label'] == "Positive"] #positive tweets

,Tweet_text,Text_cleaned,Text_lemmatized,Text_lemmatized_str,sentiment_score,Label
2,"Coronavirus outbreak: Trudeau praises ""positiv...","[coronavirus, outbreak, trudeau, praises, posi...","[coronavirus, outbreak, trudeau, praise, posit...",coronavirus outbreak trudeau praise positive c...,1,Positive
5,Gay coronavirus survivor banned from donating ...,"[gay, coronavirus, survivor, banned, donating,...","[gay, coronavirus, survivor, banned, donating,...",gay coronavirus survivor banned donating lifes...,1,Positive
9,Coronavirus: Bay Area shows progress as Califo...,"[coronavirus, bay, area, shows, progress, cali...","[coronavirus, bay, area, show, progress, calif...",coronavirus bay area show progress california ...,2,Positive
10,"Yes, we are doing an amazing job on Coronaviru...","[yes, amazing, job, coronavirus, best, world, ...","[yes, amazing, job, coronavirus, best, world, ...",yes amazing job coronavirus best world far gla...,2,Positive
16,hot girl summer has been cancelled due to coro...,"[hot, girl, summer, cancelled, due, coronaviru...","[hot, girl, summer, cancelled, due, coronaviru...",hot girl summer cancelled due coronavirus shal...,1,Positive
...,...,...,...,...,...,...
22519,"""Crisis actually brings us all much closer tog...","[crisis, actually, brings, us, much, closer, t...","[crisis, actually, brings, u, much, closer, to...",crisis actually brings u much closer together ...,1,Positive
22523,There are few better than Veena Raleigh at exp...,"[better, veena, raleigh, explaining, deaths, c...","[better, veena, raleigh, explaining, death, co...",better veena raleigh explaining death counted ...,1,Positive
22525,I would love to learn about the experience of ...,"[would, love, learn, experience, educators, co...","[would, love, learn, experience, educator, cov...",would love learn experience educator covid pan...,1,Positive
22526,My 1 year old is obsessed with @TheRealFunyuns...,"[year, old, obsessed, therealfunyuns, fritolay...","[year, old, obsessed, therealfunyuns, fritolay...",year old obsessed therealfunyuns fritolay get ...,2,Positive


In [157]:
df.loc[df['Label'] == "Negative"] #negative tweets

,Tweet_text,Text_cleaned,Text_lemmatized,Text_lemmatized_str,sentiment_score,Label
3,NOOOOOOO! No corona virus vaccine has EVER bee...,"[noo, corona, virus, vaccine, ever, safe, kill...","[noo, corona, virus, vaccine, ever, safe, kill...",noo corona virus vaccine ever safe kill need v...,-3,Negative
4,#HESA is meeting via videoconference today at ...,"[hesa, meeting, via, videoconference, today, p...","[hesa, meeting, via, videoconference, today, p...",hesa meeting via videoconference today pm et w...,-1,Negative
6,#covid19 #coronavirus #china #hackers #researc...,"[covid, coronavirus, china, hackers, research,...","[covid, coronavirus, china, hacker, research, ...",covid coronavirus china hacker research fbi ac...,-2,Negative
7,China has experienced a 60% decline in VC inve...,"[china, experienced, decline, vc, investments,...","[china, experienced, decline, vc, investment, ...",china experienced decline vc investment since ...,-1,Negative
11,Coronavirus hasn’t even killed half of this. C...,"[coronavirus, even, killed, half, crazy, peopl...","[coronavirus, even, killed, half, crazy, peopl...",coronavirus even killed half crazy people like...,-1,Negative
...,...,...,...,...,...,...
22504,#Angolan authorities have isolated a suburb of...,"[angolan, authorities, isolated, suburb, capit...","[angolan, authority, isolated, suburb, capital...",angolan authority isolated suburb capital luan...,-1,Negative
22509,COVID19 risk assessments as prescribed should ...,"[covid, risk, assessments, prescribed, compuls...","[covid, risk, assessment, prescribed, compulso...",covid risk assessment prescribed compulsory af...,-1,Negative
22521,My Hypothesis: The same way blood is filtered ...,"[hypothesis, way, blood, filtered, kidneydialy...","[hypothesis, way, blood, filtered, kidneydialy...",hypothesis way blood filtered kidneydialysis t...,-2,Negative
22522,Tip: The Fed @stevenmnuchin1 is not going to b...,"[tip, fed, stevenmnuchin, going, bail, politic...","[tip, fed, stevenmnuchin, going, bail, politic...",tip fed stevenmnuchin going bail politicans ba...,-5,Negative


In [158]:
df.loc[df['Label'] == "Neutral"] #Neutral tweets

,Tweet_text,Text_cleaned,Text_lemmatized,Text_lemmatized_str,sentiment_score,Label
0,Georgia may need to dip deep in its reserve fu...,"[georgia, may, need, dip, deep, reserve, fund,...","[georgia, may, need, dip, deep, reserve, fund,...",georgia may need dip deep reserve fund balance...,0,Neutral
1,Vatican: Migrants Are the ‘True Victims’ of th...,"[vatican, migrants, true, victims, coronavirus...","[vatican, migrant, true, victim, coronavirus, ...",vatican migrant true victim coronavirus pandemic,0,Neutral
8,A10: A money lesson I learned from the coronav...,"[money, lesson, learned, coronavirus, pandemic...","[money, lesson, learned, coronavirus, pandemic...",money lesson learned coronavirus pandemic emer...,0,Neutral
14,There are coronavirus answers in our : https:/...,"[coronavirus, answers, via, nlofholm]","[coronavirus, answer, via, nlofholm]",coronavirus answer via nlofholm,0,Neutral
15,"Solitary, Brawls, No Teachers: Coronavirus Mak...","[solitary, brawls, teachers, coronavirus, make...","[solitary, brawl, teacher, coronavirus, make, ...",solitary brawl teacher coronavirus make juveni...,0,Neutral
...,...,...,...,...,...,...
22518,about covid19 update pakistan??,"[covid, update, pakistan]","[covid, update, pakistan]",covid update pakistan,0,Neutral
22520,#COVID19 has created an unprecedented opportun...,"[covid, created, unprecedented, opportunity, s...","[covid, created, unprecedented, opportunity, s...",covid created unprecedented opportunity study ...,0,Neutral
22524,Who wants to go for bbq next week with me once...,"[wants, go, bbq, next, week, parks, open, covi...","[want, go, bbq, next, week, park, open, covid,...",want go bbq next week park open covid toronto ...,0,Neutral
22528,This is the article you should read today. #CO...,"[article, read, today, covid, homelessness, nyc]","[article, read, today, covid, homelessness, nyc]",article read today covid homelessness nyc,0,Neutral


In [160]:
df.to_csv("Preprocessed_labelled_dataset.csv")